<h1><center>HuBMAP: Complete Understanding and EDA | W&B</center></h1>
                                                      
<center><img src = "https://hubmapconsortium.org/wp-content/uploads/2019/01/HuBMAP-Retina-Logo-Color.png" width = "750" height = "500"/></center>                                                                          

<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Contents</center></h2>

> | S.No       |                   Heading                |
> | :------------- | :-------------------:                |         
> |  01 |  [**Competition Overview**](#competition-overview)  |                   
> |  02 |  [**Libraries**](#libraries)                        |  
> |  03 |  [**Global Config**](#global-config)                |
> |  04 |  [**Weights and Biases**](#weights-and-biases)      |
> |  05 |  [**Load Datasets**](#load-datasets)                |
> |  06 |  [**References**](#references)                      |

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:maroon; border:0; color:white' role="tab" aria-controls="home"><center>If you find this notebook useful, do give me an upvote, it helps to keep up my motivation. This notebook will be updated frequently so keep checking for furthur developments.</center></h3>

<a id="libraries"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Libraries</center></h2>

In [ ]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tifffile as tiff 
from tqdm.auto import tqdm

plt.style.use("Solarize_Light2")

# Wandb Login
import wandb
wandb.login()

<a id="global-config"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Global Config</center></h2>

In [ ]:
class config:
    BASE_PATH = "../input/hubmap-organ-segmentation/"
    TRAIN_PATH = os.path.join(BASE_PATH, "train")

# wandb config
WANDB_CONFIG = {
     'competition': 'HuBMAP', 
              '_wandb_kernel': 'neuracort'
    }

# Initialize W&B
run = wandb.init(
    project='hubmap-organ-segmentation', 
    config= WANDB_CONFIG
)

<a id="weights-and-biases"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Weights and Biases</center></h2>

<center><img src = "https://i.imgur.com/1sm6x8P.png" width = "750" height = "500"/></center>        

**Weights & Biases** is the machine learning platform for developers to build better models faster.

You can use W&B's lightweight, interoperable tools to

- quickly track experiments,
- version and iterate on datasets,
- evaluate model performance,
- reproduce models,
- visualize results and spot regressions,
- and share findings with colleagues.
  
Set up W&B in 5 minutes, then quickly iterate on your machine learning pipeline with the confidence that your datasets and models are tracked and versioned in a reliable system of record.

In this notebook I will use Weights and Biases's amazing features to perform wonderful visualizations and logging seamlessly.

<a id="load-datasets"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Load Datasets</center></h2>

## **<span style="color:orange;">Train Dataset</span>** 

In [ ]:
df = pd.read_csv(
    os.path.join(config.BASE_PATH, "train.csv")
)
df.head()

In [ ]:
%%capture
wandb.log({"df_train": df})

### [Interactive W&B Table for Dataframe $\rightarrow$](https://wandb.ai/ishandutta/hubmap-organ-segmentation/runs/3j0orhhe?workspace=user-ishandutta)
  
The above gif shows the following features you can seamlessly use on wandb - 
- Interactively looking at the dataframe
- Groupby based on columns and visualize the results
- Select and deselect columns
- Sorting columns in Ascending and Descending
  
[![Animation.gif](https://iili.io/hDpaUJ.gif)]

<a id="image-and-mask-single-sample"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Image and Mask Single Sample</center></h2>

## **<span style="color:orange;">Actual Image</span>** 

In [ ]:
img_id_1 = 10044
img_1 = tiff.imread(config.BASE_PATH + "train_images/" + str(img_id_1) + ".tiff")
print(img_1.shape)

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(img_1)
plt.axis("off")
wandb.log({"Image Sample 1": plt})

## **<span style="color:orange;">Mask</span>** 

In [ ]:
# https://www.kaggle.com/paulorzp/rle-functions-run-length-encode-decode
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
 
def rle2mask(mask_rle, shape=(1600,256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
mask_1 = rle2mask(df[df["id"]==img_id_1]["rle"].iloc[-1], (img_1.shape[1], img_1.shape[0]))
mask_1.shape

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(mask_1, cmap='coolwarm', alpha=0.5)
plt.axis("off")
wandb.log({"Mask Sample 1": plt})

## **<span style="color:orange;">Combining Image with Mask</span>** 

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_1)
plt.imshow(mask_1, cmap='coolwarm', alpha=0.5)
plt.axis("off")
wandb.log({"Image with Mask Sample 1": plt})

---

<a id="interactive-visualization-of-images-with-masks"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Interactive Visualization of Images with Masks</center></h2>

In [ ]:
image_ids = df.id
image_files = glob.glob(config.BASE_PATH + "/train_images/*")

In [ ]:
def save_table(table_name):
    table = wandb.Table(columns=['Id', 'Image', 'Mask', 'Image with Mask'], allow_mixed_types = True)

    for id, img in tqdm(zip(image_ids, image_files), total = len(image_ids)):

        img = tiff.imread(img)
        mask = rle2mask(df[df["id"]==id]["rle"].iloc[-1], (img.shape[1], img.shape[0]))
        
        plt.figure(figsize=(10,10))
        plt.axis("off")
        plt.imshow(img)
        plt.imshow(mask, cmap='coolwarm', alpha=0.5)
        plt.savefig("./image.jpg")
        plt.close()

        table.add_data(
            id, 
            wandb.Image(img), 
            wandb.Image(mask),
            wandb.Image(cv2.cvtColor(cv2.imread("./image.jpg"), cv2.COLOR_BGR2RGB))
        )

    wandb.log({table_name : table})
     
save_table("Images and Masks Record")

### [Interactive W&B Table with all Images and their Masks $\rightarrow$](https://wandb.ai/ishandutta/hubmap-organ-segmentation/runs/3t9d7ioa?workspace=user-ishandutta)
  
Users can easily load all the images in the form of wandb tables and view as well as compare the results.
  
[![Animation.gif](https://iili.io/hDmNwB.gif)](https://iili.io/hDmNwB.gif)

<a id="metadata-analysis"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Metadata Analysis</center></h2>

## **<span style="color:orange;">Organ</span>** 

In [ ]:
plt.figure(figsize=(15, 5))
g = sns.countplot(data=df, x="organ", palette=sns.color_palette("Set2", 8))
g.set_title("Organ Counts", color = "black")

## **<span style="color:orange;">Data Source</span>** 

In [ ]:
plt.figure(figsize=(15, 5))
g = sns.countplot(data=df, x="data_source", palette=sns.color_palette("Set2", 8))
g.set_title("Data Source", color = "black")

## **<span style="color:orange;">Age</span>** 

In [ ]:
plt.figure(figsize=(15, 5))
g = sns.histplot(data=df, x="age", palette=sns.color_palette("Set2", 8))
g.set_title("Age", color = "black")

## **<span style="color:orange;">Sex</span>** 

In [ ]:
plt.figure(figsize=(15, 5))
g = sns.histplot(data=df, x="sex", palette=sns.color_palette("Set2", 8))
g.set_title("Sex", color = "black")

<a id="references"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>References</center></h2>

>- [HuBMAP - Exploratory Data Analysis](https://www.kaggle.com/code/ihelon/hubmap-exploratory-data-analysis)
>- [HuBMAP Let's Visualize and Understand Dataset](https://www.kaggle.com/code/nayuts/hubmap-let-s-visualize-and-understand-dataset)

<h1><center>More Plots coming soon!</center></h1>
                                                      
<center><img src = "https://static.wixstatic.com/media/5f8fae_7581e21a24a1483085024f88b0949a9d~mv2.jpg/v1/fill/w_934,h_379,al_c,q_90/5f8fae_7581e21a24a1483085024f88b0949a9d~mv2.jpg" width = "750" height = "500"/></center> 

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:maroon; border:0; color:white' role="tab" aria-controls="home"><center>If you find this notebook useful, do give me an upvote, it helps to keep up my motivation. This notebook will be updated frequently so keep checking for furthur developments.</center></h3>